In [43]:
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.firefox_profile import FirefoxProfile
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.firefox import GeckoDriverManager
import pandas as pd
import time

In [44]:
PROFILE_PATH = r"/Users/gloria/Library/Application Support/Firefox/Profiles/4czc69sv.default-release"
EXCEL_FILE = "list_1.xlsx"
TWITTER_COL = "twitter"

SCROLL_PAUSE = 2   
DETAIL_PAUSE = 3   
SCROLL_ROUNDS = 20

In [45]:
accounts = pd.read_excel(EXCEL_FILE)[TWITTER_COL].dropna().tolist()
print(f"共读取 {len(accounts)} 个账号")

共读取 348 个账号


In [46]:
profile = FirefoxProfile(PROFILE_PATH)
options = Options()
options.add_argument("--width=1400")
options.add_argument("--height=900")
options.profile = profile

driver = webdriver.Firefox(
    service=Service(GeckoDriverManager().install()),
    options=options
)

In [ ]:
for account in accounts:
    search_url = f"https://x.com/search?q=true%20OR%20real%20(to%3A{account})%20(%40grok)%20lang%3Aen&src=typed_query&f=top"
    driver.get(search_url)
    print(f"\n=== 打开搜索页面: {account} ===")

    try:
        WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.TAG_NAME, "article"))
        )
    except:
        print(f"账号 {account} 页面未加载出推文")
        continue

    last_height = driver.execute_script("return document.body.scrollHeight")
    for _ in range(SCROLL_ROUNDS):
        driver.find_element(By.TAG_NAME, "body").send_keys(Keys.END)
        time.sleep(SCROLL_PAUSE)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
    print(" 页面滚动完成，所有推文已加载到 DOM")

    all_articles = driver.find_elements(By.TAG_NAME, "article")
    tweet_links = []
    for article in all_articles:
        try:
            link = article.find_element(By.CSS_SELECTOR, "a[href*='/status/']").get_attribute("href")
            tweet_links.append(link)
        except:
            continue

    print(f"共收集到 {len(tweet_links)} 条推文链接")

    for idx, link in enumerate(tweet_links):
        try:
            driver.get(link)
            print(f"已进入第 {idx+1} 条推文页面 ({account})")
            time.sleep(DETAIL_PAUSE)  
            driver.back()
            print(f"已返回列表页 ({account})")
            time.sleep(2)
        except Exception as e:
            print(f"跳过第 {idx+1} 条推文（错误）：{e}")
            try:
                driver.back()
            except:
                pass
            time.sleep(2)

print("\n全部账号处理完成")
driver.quit()


=== 打开搜索页面: jahimes ===
账号 jahimes 页面未加载出推文

=== 打开搜索页面: Yahoo ===
账号 Yahoo 页面未加载出推文

=== 打开搜索页面: WhipKClark ===
 页面滚动完成，所有推文已加载到 DOM
共收集到 12 条推文链接
已进入第 1 条推文页面 (WhipKClark)
已返回列表页 (WhipKClark)
已进入第 2 条推文页面 (WhipKClark)
跳过第 2 条推文（错误）：Message: Browsing context has been discarded
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:202:5
NoSuchWindowError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:782:5
assert.that/<@chrome://remote/content/shared/webdriver/Assert.sys.mjs:581:13
assert.open@chrome://remote/content/shared/webdriver/Assert.sys.mjs:169:4
GeckoDriver.prototype.goBack@chrome://remote/content/marionette/driver.sys.mjs:1268:39
dispatch@chrome://remote/content/marionette/server.sys.mjs:318:40
execute@chrome://remote/content/marionette/server.sys.mjs:289:16
onPacket/<@chrome://remote/content/marionette/server.sys.mjs:262:20
onPacket@chrome://remote/content/marionette/ser

In [ ]:
except Exception as e:
    print(f"\n出现错误: {e}")
    print("浏览器保持打开状态，请手动检查当前页面")
else:
    print("\n全部账号处理完成")
    driver.quit()